In [1]:
%load_ext autoreload
%autoreload 2

from klampt import *
from IPython.display import clear_output
import ipywidgets as widgets
import ipyklampt
import math
import RobotControllerEmulator

#the emulator and visualization need a world
world = WorldModel()
res = world.readFile("data/ur5Cart.xml")
if not res:
    raise RuntimeError("Unable to load file")

ee_driver = 6
    
controlFuncs = dict()

def loadActualReadings(name):
    av_file = open("control_calibration_data/" + name + "_a.txt")
    actual_values = []
    for line in av_file:
        splits = line[:-1].split(' ')
        for i in range(len(splits)):
            splits[i] = float(splits[i])
        actual_values.append(splits)
    av_file.close()
    return actual_values

def controlFunc1(time):
    if time <= 1.2:
        # no command for the first 1.2 seconds
        pass
    else:
        return [0,-math.pi/2,math.pi/4,-math.pi/2,0,0,1]

def controlFunc2(time):
    if time <= 1.2:
        # no command for the first 1.2 seconds
        pass
    else:
        return [math.pi/4,-math.pi/2,math.pi/4,-math.pi/2,0,0,0]

def controlFunc3(time):
    if time <= 1.2:
        # no command for the first 1.2 seconds
        pass
    else:
        t=time-1.2
        if t<0.1:
            return [0,-math.pi/2,math.pi/4,-math.pi/2,0,0,1]
        else:
            return [0,-math.pi/2,0,-math.pi/2,0,0,0]

def controlFunc4(time):
    if time <= 1.2:
        # no command for the first 1.2 seconds
        pass
    else:
        t=time-1.2
        if t<0.1:
            return [math.pi/4,-math.pi/2,math.pi/4,-math.pi/2,0,0,0]
        else:
            return [0,-math.pi/2,0,-math.pi/2,0,0,0]
        
def controlFunc5(time):
    if time <= 1.2:
        # no command for the first 1.2 seconds
        pass
    else:
        t=time-1.2
        q1=0.7*math.sin(t)
        q3=0.7*math.sin(t)
        q7=abs(math.sin(0.5*t))
        return [q1,-math.pi/2,q3,-math.pi/2,0,0,q7]

def controlFunc6(time):
    if time <= 1.2:
        # no command for the first 1.2 seconds
        pass
    else:
        t=time-1.2
        q1=0.7*math.cos(2*t)
        q3=0.7*math.cos(2*t)
        q7=abs(math.cos(t))
        return [q1,-math.pi/2,q3,-math.pi/2,0,0,q7]

controlFuncs['test1'] = controlFunc1
controlFuncs['test2'] = controlFunc2
controlFuncs['test3'] = controlFunc3
controlFuncs['test4'] = controlFunc4
controlFuncs['test5'] = controlFunc5
controlFuncs['test6'] = controlFunc6

#how often to update the visualization during testing
VIS_UPDATE_FREQUENCY = 10

def runTest(testName):
    """Return value is a tuple (times,qact,dqact,qcmd,qsim,dqsim)"""
    reload(RobotControllerEmulator)
    testActual = loadActualReadings(testName)

    q_ur5_init = testActual[0][1:7]
    q_gripper_init = testActual[0][8]
    q = world.robot(0).getConfig()
    for i in range(6):
        q[i+1] = q_ur5_init[i]
    world.robot(0).setConfig(q)
    world.robot(0).driver(ee_driver).setValue(q_gripper_init)
    emulator = RobotControllerEmulator.UR5WithGripperController(world)

    #traces of the commanded configuration, sensed configuration, and sensed velocity
    qcmd_trace = []
    qsim_trace = []
    dqsim_trace = []

    tprev = 0
    for i in range(len(testActual)):
        t = testActual[i][0]
        q = emulator.getConfig()
        dq = emulator.getVelocity()
        time = emulator.getCurrentTime()
        if t > time:
            qcmd = controlFuncs[testName](time)
            if qcmd:
                emulator.setConfig(qcmd) 
            if abs((t-time) - emulator.dt) > 1e-5:
                print "Timing mismatch between emulated controller frequency and the file"
                print emulator.dt,"vs",t-time
                emulator.dt = t - time
            emulator.advance()
            if i%VIS_UPDATE_FREQUENCY==0:
                kvis.update()
        else:
            qcmd = None
        qcmd_trace.append(qcmd)
        qsim_trace.append(q)
        dqsim_trace.append(dq)
    print "done"
    times = [v[0] for v in testActual]
    qact_trace = [v[1:8] for v in testActual]
    dqact_trace = [v[8:15] for v in testActual]
    return times,qact_trace,dqact_trace,qcmd_trace,qsim_trace,dqsim_trace


In [2]:
#plotting code
%matplotlib notebook
import matplotlib.pyplot as plt

def plotone(ax,times,qact,qcmd,qsim,label):
    if qact is not None:
        ax.plot(times, qact, 'k-')
    if qcmd is not None:
        ax.plot(times, qcmd, 'g-')
    if qsim is not None:
        ax.plot(times, qsim, 'r-')
    ax.set_ylim(-2,2)
    ax.set_ylabel(label)
    ax.set_xlabel('t')
    ax.grid()

def plotall(times,qact,qcmd,qsim):
    plt.figure(figsize=(8,6))
    for i in range(7):
        ax = plt.subplot(3,3, (i+1 if i!=6 else 9))
        label = "q"+str(i)
        if i == 6:
            label = "gripper"
        plotone(ax,times,[v[i] for v in qact],[(v[i] if v is not None else float('inf')) for v in qcmd],[v[i] for v in qsim],label)
    plt.legend(['Physical Robot Positions','Commanded Positions','Simulated Positions'], loc=[-3,0.25])
    plt.tight_layout()
    plt.show()

def plotall_velocities(times,dqact,dqsim):
    plt.figure(figsize=(8,6))
    for i in range(7):
        ax = plt.subplot(3,3, (i+1 if i!=6 else 9))
        label = "dq"+str(i)
        if i == 6:
            label = "dgripper"
        plotone(ax,times,[v[i] for v in dqact],None,[v[i] for v in dqsim],label)
    plt.legend(['Physical Robot Velocities','Simulated Velocities'], loc=[-3,0.25])
    plt.tight_layout()
    plt.show()


In [3]:
from ipywidgets import interact

kvis = ipyklampt.KlamptWidget(world,width=600,height=400)
kvis.set_camera({u'near': 0.2, u'target': {u'y': 1.1188322004142854, u'x': 0.042176605196346695, u'z': 0.009329657831685366}, u'far': 1000, u'position': {u'y': 1.6142988723996625, u'x': 1.5814619610767169, u'z': -0.03643442712963929}, u'up': {u'y': 1, u'x': 0, u'z': 0}})

#If you'd like to show the print output, comment out this line
clear_output()
display(kvis)

dropdown = widgets.Dropdown(
    options=sorted(controlFuncs.keys()),
    value='test1',
    description='Test case:',
    disabled=False,
)
def doRunTest(testName):
    times,qact,dqact,qcmd,qsns,dqsns = runTest(testName)
    plotall(times,qact,qcmd,qsns)
    plotall_velocities(times,dqact,dqsns)
interact(doRunTest,testName=dropdown)

S2xhbXB0V2lkZ2V0KGNhbWVyYT17dSdmYXInOiAxMDAwLCB1J3Bvc2l0aW9uJzoge3UneSc6IDEuNjE0Mjk4ODcyMzk5NjYyNSwgdSd4JzogMS41ODE0NjE5NjEwNzY3MTY5LCB1J3onOiAtMC7igKY=


aW50ZXJhY3RpdmUoY2hpbGRyZW49KERyb3Bkb3duKGRlc2NyaXB0aW9uPXUnVGVzdCBjYXNlOicsIG9wdGlvbnM9KCd0ZXN0MScsICd0ZXN0MicsICd0ZXN0MycsICd0ZXN0NCcsICd0ZXN0NSfigKY=


<function __main__.doRunTest>

## Written answers
Problem A